# Run LLM Locally : A Step-by-Step Guide

## Introduction 🌟

We’ll leverage tools like GPT4All and llama-cpp-python to set up and run LLMs on your local laptop for enhanced privacy and cost-effectiveness.

## Set Up Your Environment 🛠️

In [ ]:
!pip install --upgrade llama-cpp-python langchain gpt4all llama-index sentence-transformers

## Run LLM Locally 🏡: 1st attempt

>> Download the Model 🚀

In [ ]:
!huggingface-cli download TheBloke/Llama-2–7b-Chat-GGUF llama-2–7b-chat.Q5_K_S.gguf — local-dir . — local-dir-use-symlinks False

>> Load and Use the Model 🚀

In [ ]:
from llama_cpp import Llama

In [ ]:
# Load the model
llm = Llama(model_path='/content/llama-2-7b-chat.Q5_K_S.gguf')

Now, you can complete sentences using your LLM:

In [ ]:
sentence = "The capital of Tunisia is "
response = llm(sentence)
print(response['choices'][0]['text'])

## Manage Personal Data 🗃️

---

For simplicity, we’ll use a text file as our data source. Later, you can copy and paste information from your calendar, news, articles, books, etc. Here’s an example of a mock calendar:

---


In [ ]:
%%writefile ./data.txt
# data.txt

11 Jan - Go to the movie
12 Jan - Have a dinner with family
13 Jan - Go to the birthday party
14 Jan - Go to the dentist
15 Jan - Finish reviewing a git pull request

## Create Necessary Components 🧩

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index import PromptHelper
from llama_index.node_parser import SentenceSplitter

In [ ]:
# An embedding model used to structure text into representations
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

# PromptHelper can help deal with LLM context window and token limitations
prompt_helper = PromptHelper(context_window=2048)

# SentenceSplitter used to split our data into multiple chunks
# Only a number of relevant chunks will be retrieved and fed into LLMs
node_parser = SentenceSplitter(chunk_size=300, chunk_overlap=20)

## Setting Up LlamaCPP for Embedding and Sentence Handling

In [ ]:
# Code for embedding, prompt helper, and sentence splitter
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=None,#model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path='llama-2-7b-chat.Q5_K_S.gguf',
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},# -1 for selecting all cores
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

## Context Service 📦

In [ ]:
from llama_index import ServiceContext

In [ ]:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    prompt_helper=prompt_helper,
    node_parser=node_parser
)

## Build an Index and Query Engine 📊

In [ ]:
from llama_index import SimpleDirectoryReader
from llama_index import VectorStoreIndex

In [ ]:
# Load data.txt into a document
document = SimpleDirectoryReader(input_files=['./data.txt']).load_data()

# Process data (chunking, embedding, indexing) and store them
index = VectorStoreIndex.from_documents(
    document, service_context=service_context)

# Build a query engine from the index
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query('Give me my calendar.')
print(response)

In [ ]:
response = query_engine.query("What's the plan for 13 Jan?")
print(response)

# Next steps: Summarize a book 📚

# Conclusion 🎉

Congratulations! You’ve successfully created your personal assistant.

Sources :

GitHub : https://github.com/fedihamdi

LlamaIndex : https://docs.llamaindex.ai/en/stable/examples/llm/llama_2_llama_cpp.html

HuggingFace : https://huggingface.co/docs/huggingface_hub/guides/cli

Gpt4all : https://gpt4all.io/index.html